In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import gc
from accelerate import Accelerator # Importation ajoutée

# Initialiser l'accélérateur explicitement
accelerator = Accelerator()

# Charger les données préparées
print("Chargement des données...")
train_df = pd.read_csv('/content/drive/MyDrive/train_data_distilroberta.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test_data_distilroberta.csv')

# --- Stratégie : Réduire la taille des datasets pour un test rapide ---
# Décommentez et ajustez le 'frac' pour utiliser un pourcentage des données
# print("Option: Réduction de la taille des datasets (ex: 10%)...")
# train_df = train_df.sample(frac=0.1, random_state=42)
# test_df = test_df.sample(frac=0.1, random_state=42)
# print(f"Nouvelle taille du train_df: {len(train_df)}, test_df: {len(test_df)}")

# Convertir en datasets Hugging Face
print("Conversion en datasets Hugging Face...")
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Supprimer les dataframes pandas pour libérer de la mémoire
del train_df
del test_df
gc.collect() # Forcer le garbage collection

# Charger le tokenizer et le modèle
print("Chargement du modèle et du tokenizer...")
# --- Stratégie : Utiliser un modèle plus petit ---
# model_name = "distilroberta-base" # Alternative plus légère
# model_name = "microsoft/deberta-v3-small" # Autre alternative (vérifier num_labels et config)
model_name = "j-hartmann/emotion-english-distilroberta-base" # Modèle remplacé par DistilRoBERTa pour la classification d'émotions [2]
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7) # num_labels=7 correspond à ce modèle [2]

# Fonction de tokenization
def tokenize_function(examples):
    # --- Stratégie : Réduire max_length si possible ---
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64) # max_length réduite à 64

# Appliquer la tokenization
print("Tokenisation des données...")
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"]) # remove_columns directement
tokenized_test = test_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Supprimer les datasets bruts pour libérer de la mémoire
del train_dataset
del test_dataset
gc.collect()

# Renommer la colonne "label" en "labels" (requis par le Trainer)
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_test = tokenized_test.rename_column("label", "labels")

# Définir le format des données
# tokenized_train.set_format("torch")  # Lignes supprimées car Accelerator s'en charge implicitement
# tokenized_test.set_format("torch")   # Lignes supprimées

# Fonction de calcul des métriques
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    _, _, class_f1, _ = precision_recall_fscore_support(labels, preds, average=None, zero_division=0)
    class_metrics = {f"f1_class_{i}": score for i, score in enumerate(class_f1)}
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall, **class_metrics}

# Configuration de l'entraînement avec des paramètres réduits pour économiser les ressources et voir les métriques rapidement
print("Configuration de l'entraînement pour économiser les ressources et obtenir un aperçu rapide des métriques...")
training_args = TrainingArguments(
    output_dir="./results_distilroberta_friendly", # Chemin de sortie mis à jour
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # --- Stratégie : Taille de batch très réduite ---
    per_device_eval_batch_size=16,   # --- Stratégie : Taille de batch très réduite ---
    # gradient_accumulation_steps=2, # Optionnel: pour simuler un batch plus grand si besoin
    num_train_epochs=3,  # --- Stratégie : Nombre d'époques minimal ---
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    report_to="none", # ou ["tensorboard", "wandb"] si vous utilisez ces outils pour le logging visuel
    run_name="emotion_classification_distilroberta_friendly_preview", # Nom d'exécution mis à jour
    logging_strategy="steps",
    logging_steps=2,
    fp16=torch.cuda.is_available(),  # Activer fp16 uniquement si un GPU est disponible et compatible
)

# Initialiser le Trainer avec early stopping
print("Initialisation du Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)] # Patience réduite
)

# Lancer l'entraînement
print("Début de l'entraînement...")
try:
    trainer.train()
except RuntimeError as e:
    if "out of memory" in str(e).lower(): # Comparaison insensible à la casse
        print("Erreur CUDA Out of Memory détectée. Essayez de réduire davantage la taille du batch,")
        print("la longueur de la séquence, ou utilisez un modèle plus petit.")
        print("Libération de la mémoire cache CUDA...")
        if torch.cuda.is_available():
            torch.cuda.empty_cache() # --- Stratégie : Libérer la mémoire GPU ---
    else:
        raise e # Relancer d'autres erreurs

# Évaluer le modèle (optionnel si l'entraînement a échoué)
model_path = "./distilroberta-emotion-resource-friendly" # Chemin de sauvegarde mis à jour

if hasattr(trainer, 'state') and trainer.state.best_model_checkpoint is not None:
    print("Évaluation du modèle...")
    eval_results = trainer.evaluate()
    print(f"Résultats d'évaluation: {eval_results}")

    # Sauvegarder le modèle
    # Le modèle est déjà sauvegardé par le Trainer via save_steps et load_best_model_at_end
    # Si vous voulez sauvegarder explicitement le meilleur modèle à la fin, vous pouvez utiliser:
    # trainer.save_model(model_path)
    # tokenizer.save_pretrained(model_path)
    # Ou, si vous voulez sauvegarder le modèle final quel qu'il soit (pas forcément le meilleur)
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)
    print(f"Modèle sauvegardé à {model_path}")
else:
    print("L'entraînement n'a pas pu se terminer correctement ou aucun meilleur modèle n'a été trouvé.")
    # Tentative de sauvegarde du modèle actuel s'il existe, au cas où
    try:
        model.save_pretrained(model_path)
        tokenizer.save_pretrained(model_path)
        print(f"Modèle actuel (potentiellement non optimal) sauvegardé à {model_path}")
    except Exception as e_save:
        print(f"Impossible de sauvegarder le modèle actuel: {e_save}")


# --- Stratégie : Libérer la mémoire du modèle et des datasets tokenisés après usage ---
del model
del tokenized_train
del tokenized_test
# del trainer # Supprimer l'objet Trainer aussi pour libérer sa mémoire
if 'trainer' in locals(): del trainer # Vérifier si trainer existe avant de le supprimer
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

# Tester le modèle sur quelques exemples (recharger si nécessaire, ou utiliser le modèle sauvegardé)
# Pour ce test, nous allons supposer que le modèle a été sauvegardé et le recharger
# Cela garantit que nous n'utilisons pas le modèle en mémoire qui vient d'être supprimé.
print("\nTest du modèle sur quelques exemples (après rechargement):")
try:
    # Il est important que model_path ici corresponde à celui où le meilleur modèle a été sauvegardé
    # Si load_best_model_at_end=True, le Trainer recharge le meilleur modèle à la fin.
    # Le model_path utilisé ci-dessous doit pointer vers cette sauvegarde.
    reloaded_model = AutoModelForSequenceClassification.from_pretrained(model_path)
    reloaded_tokenizer = AutoTokenizer.from_pretrained(model_path)
    if torch.cuda.is_available():
        reloaded_model.to(accelerator.device) # Utiliser accelerator.device

    # Recharger une petite portion des données de test pour l'exemple
    test_examples_df_sample = pd.read_csv('/content/drive/MyDrive/test_data_distilroberta.csv').sample(5, random_state=42)
    test_examples = test_examples_df_sample["text"].tolist()
    true_labels = test_examples_df_sample["label"].tolist()

    emotion_mapping = {0: "anger", 1: "disgust", 2: "fear", 3: "joy", 4: "neutral", 5: "sadness", 6: "surprise"}

    for i, text in enumerate(test_examples):
        inputs = reloaded_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=64) # Utiliser la même max_length
        if torch.cuda.is_available():
            inputs = {k: v.to(reloaded_model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = reloaded_model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = outputs.logits.argmax(-1).item()
        confidence = probs[0][predicted_class].item()
        true_emotion = emotion_mapping[true_labels[i]]
        predicted_emotion = emotion_mapping[predicted_class]
        print(f"Texte: {text}")
        print(f"Émotion réelle: {true_emotion}")
        print(f"Émotion prédite: {predicted_emotion} (confiance: {confidence:.4f})")
        print("-" * 80)

except NameError: # Si model_path n'a pas été défini (par ex. si l'entraînement a échoué très tôt)
    print(f"Variable 'model_path' non définie. Le modèle n'a probablement pas été sauvegardé, test annulé.")
except OSError: # Si le chemin du modèle n'existe pas
    print(f"Le modèle n'a pas été trouvé à '{model_path}'. Assurez-vous que le chemin est correct et que le modèle a été sauvegardé. Test annulé.")
except Exception as e:
    print(f"Erreur lors du test du modèle rechargé: {e}")

print("Script terminé.")


Chargement des données...
Conversion en datasets Hugging Face...
Chargement du modèle et du tokenizer...


tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

<ipython-input-10-a1bfd2adae37>:102: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6
50,1.562400,1.402213,0.517544,0.506944,0.509859,0.517544,0.487805,0.000000,0.428571,0.521739,0.606965,0.416667,0.333333
100,0.992300,1.325116,0.539474,0.503689,0.495233,0.539474,0.493506,0.000000,0.000000,0.536585,0.646288,0.347826,0.320000
150,1.381700,1.378866,0.539474,0.526282,0.529453,0.539474,0.554217,0.000000,0.333333,0.562500,0.628866,0.357143,0.320000
200,0.888800,1.374544,0.570175,0.554009,0.551870,0.570175,0.590909,0.000000,0.333333,0.559140,0.673469,0.347826,0.370370
250,0.424200,1.368496,0.565789,0.549930,0.546793,0.565789,0.577778,0.000000,0.333333,0.561798,0.680203,0.272727,0.357143
300,0.701000,1.387894,0.578947,0.559182,0.552437,0.578947,0.597701,0.000000,0.315789,0.558140,0.696078,0.285714,0.357143


model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Évaluation du modèle...


Résultats d'évaluation: {'eval_loss': 1.3878941535949707, 'eval_accuracy': 0.5789473684210527, 'eval_f1': 0.5591817622212976, 'eval_precision': 0.5524371774451705, 'eval_recall': 0.5789473684210527, 'eval_f1_class_0': 0.5977011494252874, 'eval_f1_class_1': 0.0, 'eval_f1_class_2': 0.3157894736842105, 'eval_f1_class_3': 0.5581395348837209, 'eval_f1_class_4': 0.696078431372549, 'eval_f1_class_5': 0.2857142857142857, 'eval_f1_class_6': 0.35714285714285715, 'eval_runtime': 0.2245, 'eval_samples_per_second': 1015.629, 'eval_steps_per_second': 66.818, 'epoch': 3.0}
Modèle sauvegardé à ./distilroberta-emotion-resource-friendly

Test du modèle sur quelques exemples (après rechargement):
Texte: poverty is the greatest cause behind child labour.
Émotion réelle: anger
Émotion prédite: disgust (confiance: 0.3967)
--------------------------------------------------------------------------------
Texte: “: stop ur false propaganda
Émotion réelle: anger
Émotion prédite: anger (confiance: 0.9681)
-------

In [ ]:
# Initialiser l'accélérateur explicitement
accelerator = Accelerator()

# Charger les données préparées
print("Chargement des données...")
train_df = pd.read_csv('/content/drive/MyDrive/train_data_distilroberta.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test_data_distilroberta.csv')

# --- Stratégie : Réduire la taille des datasets pour un test rapide ---
# Décommentez et ajustez le 'frac' pour utiliser un pourcentage des données
print("Option: Réduction de la taille des datasets (ex: 10%)...")
# train_df = train_df.sample(frac=0.1, random_state=42)
# test_df = test_df.sample(frac=0.1, random_state=42)
# print(f"Nouvelle taille du train_df: {len(train_df)}, test_df: {len(test_df)}")

# Convertir en datasets Hugging Face
print("Conversion en datasets Hugging Face...")
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Supprimer les dataframes pandas pour libérer de la mémoire
del train_df
del test_df
gc.collect() # Forcer le garbage collection

# Charger le tokenizer et le modèle
print("Chargement du modèle et du tokenizer...")
# --- Stratégie : Utiliser un modèle plus petit ---
# model_name = "distilroberta-base" # Alternative plus légère
# model_name = "microsoft/deberta-v3-small" # Autre alternative (vérifier num_labels et config)
model_name = "tae898/emoberta-large" # Modèle actuel
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7)

# Fonction de tokenization
def tokenize_function(examples):
    # --- Stratégie : Réduire max_length si possible ---
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64) # max_length réduite à 64

# Appliquer la tokenization
print("Tokenisation des données...")
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"]) # remove_columns directement
tokenized_test = test_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Supprimer les datasets bruts pour libérer de la mémoire
del train_dataset
del test_dataset
gc.collect()

# Renommer la colonne "label" en "labels" (requis par le Trainer)
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_test = tokenized_test.rename_column("label", "labels")

# Définir le format des données
# tokenized_train.set_format("torch")  # Remove these lines
# tokenized_test.set_format("torch")   # Remove these lines

# Fonction de calcul des métriques
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    _, _, class_f1, _ = precision_recall_fscore_support(labels, preds, average=None, zero_division=0)
    class_metrics = {f"f1_class_{i}": score for i, score in enumerate(class_f1)}
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall, **class_metrics}

# Configuration de l'entraînement avec des paramètres réduits pour économiser les ressources et voir les métriques rapidement
print("Configuration de l'entraînement pour économiser les ressources et obtenir un aperçu rapide des métriques...")
training_args = TrainingArguments(
    output_dir="./results_resource_friendly",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # --- Stratégie : Taille de batch très réduite ---
    per_device_eval_batch_size=16,   # --- Stratégie : Taille de batch très réduite ---
    # gradient_accumulation_steps=2, # Optionnel: pour simuler un batch plus grand si besoin
    num_train_epochs=3,  # --- Stratégie : Nombre d'époques minimal ---
    weight_decay=0.01,
    eval_strategy="steps",  # MODIFIÉ: Évaluer par steps [1]
    eval_steps=50,  # MODIFIÉ: Évaluer tous les 5 steps pour un retour très rapide [1]
    save_strategy="steps",  # MODIFIÉ: Sauvegarder par steps (peut être "no" si pas de sauvegarde nécessaire pour l'aperçu) [1]
    save_steps=50,  # MODIFIÉ: Sauvegarder tous les 5 steps (ajuster si trop fréquent) [1]
    load_best_model_at_end=True, # Maintenu, mais pourrait être False pour un aperçu encore plus rapide si le rechargement n'est pas crucial
    metric_for_best_model="f1",
    push_to_hub=False,
    report_to="none", # ou ["tensorboard", "wandb"] si vous utilisez ces outils pour le logging visuel
    run_name="emotion_classification_resource_friendly_preview", # Nom d'exécution mis à jour
    logging_strategy="steps",  # MODIFIÉ: Logger par steps [1]
    logging_steps=2,  # MODIFIÉ: Logger toutes les 2 steps pour un retour quasi immédiat des métriques d'entraînement (comme la loss) [1]
    fp16=torch.cuda.is_available(),  # Activer fp16 uniquement si un GPU est disponible et compatible
)


# Initialiser le Trainer avec early stopping
print("Initialisation du Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)] # Patience réduite
)

# Lancer l'entraînement
print("Début de l'entraînement...")
try:
    trainer.train()
except RuntimeError as e:
    if "out of memory" in str(e):
        print("Erreur CUDA Out of Memory détectée. Essayez de réduire davantage la taille du batch,")
        print("la longueur de la séquence, ou utilisez un modèle plus petit.")
        print("Libération de la mémoire cache CUDA...")
        if torch.cuda.is_available():
            torch.cuda.empty_cache() # --- Stratégie : Libérer la mémoire GPU ---
    else:
        raise e # Relancer d'autres erreurs

# Évaluer le modèle (optionnel si l'entraînement a échoué)
if hasattr(trainer, 'state') and trainer.state.best_model_checkpoint is not None:
    print("Évaluation du modèle...")
    eval_results = trainer.evaluate()
    print(f"Résultats d'évaluation: {eval_results}")

    # Sauvegarder le modèle
    model_path = "./emoberta-resource-friendly"
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)
    print(f"Modèle sauvegardé à {model_path}")
else:
    print("L'entraînement n'a pas pu se terminer correctement ou aucun meilleur modèle n'a été trouvé.")

# --- Stratégie : Libérer la mémoire du modèle et des datasets tokenisés après usage ---
del model
del tokenized_train
del tokenized_test
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

# Tester le modèle sur quelques exemples (recharger si nécessaire, ou utiliser le modèle sauvegardé)
# Pour ce test, nous allons supposer que le modèle a été sauvegardé et le recharger
# Cela garantit que nous n'utilisons pas le modèle en mémoire qui vient d'être supprimé.
print("\nTest du modèle sur quelques exemples (après rechargement):")
try:
    reloaded_model = AutoModelForSequenceClassification.from_pretrained(model_path)
    reloaded_tokenizer = AutoTokenizer.from_pretrained(model_path)
    if torch.cuda.is_available():
        reloaded_model.to('cuda') # S'assurer que le modèle est sur GPU si disponible

    # Recharger une petite portion des données de test pour l'exemple
    test_examples_df_sample = pd.read_csv('/content/drive/MyDrive/test_data_distilroberta.csv').sample(5, random_state=42)
    test_examples = test_examples_df_sample["text"].tolist()
    true_labels = test_examples_df_sample["label"].tolist()

    emotion_mapping = {0: "anger", 1: "disgust", 2: "fear", 3: "joy", 4: "neutral", 5: "sadness", 6: "surprise"}

    for i, text in enumerate(test_examples):
        inputs = reloaded_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=64) # Utiliser la même max_length
        if torch.cuda.is_available():
            inputs = {k: v.to(reloaded_model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = reloaded_model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = outputs.logits.argmax(-1).item()
        confidence = probs[0][predicted_class].item()
        true_emotion = emotion_mapping[true_labels[i]]
        predicted_emotion = emotion_mapping[predicted_class]
        print(f"Texte: {text}")
        print(f"Émotion réelle: {true_emotion}")
        print(f"Émotion prédite: {predicted_emotion} (confiance: {confidence:.4f})")
        print("-" * 80)

except NameError:
    print("Le modèle n'a pas été sauvegardé ou le chemin est incorrect, test annulé.")
except Exception as e:
    print(f"Erreur lors du test du modèle rechargé: {e}")


print("Script terminé.")

Chargement des données...
Option: Réduction de la taille des datasets (ex: 10%)...
Conversion en datasets Hugging Face...
Chargement du modèle et du tokenizer...
Tokenisation des données...


Map:   0%|          | 0/912 [00:00<?, ? examples/s]

Map:   0%|          | 0/228 [00:00<?, ? examples/s]

Configuration de l'entraînement pour économiser les ressources et obtenir un aperçu rapide des métriques...
Initialisation du Trainer...


<ipython-input-3-c4a86cd3fba9>:95: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Début de l'entraînement...


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6
50,2.044900,1.542984,0.442982,0.308749,0.262110,0.442982,0.315789,0.000000,0.000000,0.000000,0.605263,0.000000,0.000000
100,1.462000,1.402932,0.504386,0.427325,0.420496,0.504386,0.485981,0.000000,0.000000,0.360656,0.655462,0.000000,0.000000
150,1.307200,1.354007,0.521930,0.497811,0.493622,0.521930,0.520000,0.000000,0.142857,0.615385,0.625000,0.086957,0.222222
200,0.903400,1.369554,0.539474,0.484816,0.485301,0.539474,0.604651,0.000000,0.000000,0.461538,0.658333,0.000000,0.222222
250,0.709600,1.360408,0.535088,0.511699,0.495485,0.535088,0.643678,0.000000,0.000000,0.549451,0.649746,0.148148,0.206897
300,0.918400,1.314942,0.548246,0.532061,0.518811,0.548246,0.625000,0.000000,0.000000,0.541176,0.679803,0.173913,0.342857


Évaluation du modèle...


Résultats d'évaluation: {'eval_loss': 1.3149422407150269, 'eval_accuracy': 0.5482456140350878, 'eval_f1': 0.532061182201911, 'eval_precision': 0.5188114269591135, 'eval_recall': 0.5482456140350878, 'eval_f1_class_0': 0.625, 'eval_f1_class_1': 0.0, 'eval_f1_class_2': 0.0, 'eval_f1_class_3': 0.5411764705882353, 'eval_f1_class_4': 0.6798029556650246, 'eval_f1_class_5': 0.17391304347826086, 'eval_f1_class_6': 0.34285714285714286, 'eval_runtime': 0.8155, 'eval_samples_per_second': 279.566, 'eval_steps_per_second': 18.393, 'epoch': 3.0}
Modèle sauvegardé à ./emoberta-resource-friendly

Test du modèle sur quelques exemples (après rechargement):
Texte: poverty is the greatest cause behind child labour.
Émotion réelle: anger
Émotion prédite: sadness (confiance: 0.4663)
--------------------------------------------------------------------------------
Texte: “: stop ur false propaganda
Émotion réelle: anger
Émotion prédite: anger (confiance: 0.9632)
------------------------------------------------

In [ ]:
import pandas as pd
import re
import numpy as np
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

# Mapping des émotions numériques vers leurs noms
mapping_emotions = {
    1: 'fear',
    2: 'anger',
    3: 'joy',
    4: 'surprise',
    5: 'sadness',
    6: 'disgust',
    7: 'neutral'
}

# Mapping pour convertir vos étiquettes (1-7) vers celles du modèle (0-6)
model_label_mapping = {
    1: 2,  # fear -> 2
    2: 0,  # anger -> 0
    3: 3,  # joy -> 3
    4: 6,  # surprise -> 6
    5: 5,  # sadness -> 5
    6: 1,  # disgust -> 1
    7: 4   # neutral -> 4
}

# Chargement du fichier CSV
print("Chargement du fichier CSV...")
df = pd.read_csv('/content/drive/MyDrive/Annotation_final.csv')

# Fonction de nettoyage du texte
def nettoyer_texte(texte):
    texte = str(texte).lower()  # conversion en minuscules
    texte = re.sub(r"http\S+|www\S+", "", texte)  # suppression des URLs
    texte = re.sub(r"@\w+|#\w+", "", texte)  # suppression des mentions et hashtags
    texte = re.sub(r"\s+", " ", texte).strip()  # suppression des espaces multiples
    return texte

# Application du prétraitement
print("Application du prétraitement...")
df['texte_nettoye'] = df['text'].apply(nettoyer_texte)

# Conversion des étiquettes au format attendu par le modèle
df['emotion_texte'] = df['emotions'].map(mapping_emotions)
df['emotion_model'] = df['emotions'].map(model_label_mapping)

# Division des données en ensembles d'entraînement et de test
print("Division des données en ensembles d'entraînement et de test...")
X_train, X_test, y_train, y_test = train_test_split(
    df['texte_nettoye'],
    df['emotion_model'],
    test_size=0.2,
    random_state=42,
    stratify=df['emotions']
)

# Création des dataframes d'entraînement et de test
train_df = pd.DataFrame({
    'text': X_train.values,
    'label': y_train.values
})

test_df = pd.DataFrame({
    'text': X_test.values,
    'label': y_test.values
})

# Affichage des informations
print("\nDistribution des émotions dans l'ensemble d'entraînement:")
print(train_df['label'].value_counts().sort_index())

print("\nDistribution des émotions dans l'ensemble de test:")
print(test_df['label'].value_counts().sort_index())

# Sauvegarde des dataframes
train_df.to_csv('/content/drive/MyDrive/train_data_distilroberta.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/test_data_distilroberta.csv', index=False)

print("\nDataframes d'entraînement et de test sauvegardés avec succès!")
print("Nombre d'exemples d'entraînement:", len(train_df))
print("Nombre d'exemples de test:", len(test_df))


Chargement du fichier CSV...
Application du prétraitement...
Division des données en ensembles d'entraînement et de test...

Distribution des émotions dans l'ensemble d'entraînement:
label
0    147
1     37
2     42
3    174
4    387
5     52
6     73
Name: count, dtype: int64

Distribution des émotions dans l'ensemble de test:
label
0    37
1     9
2    10
3    44
4    97
5    13
6    18
Name: count, dtype: int64

Dataframes d'entraînement et de test sauvegardés avec succès!
Nombre d'exemples d'entraînement: 912
Nombre d'exemples de test: 228
